In [1]:
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pathlib

In [143]:
def list_exp_csv(csv_dir, exp_name, pre_gan, metrics):
    if isinstance(csv_dir, str):
        csv_dir = pathlib.Path(csv_dir)
    csv_list = []
    for metric in metrics:
        csv_list.append(list(csv_dir.glob('**/' + exp_name + '/*' + pre_gan + '*' + metric + '*.csv')))
    csv_list = [item for sublist in csv_list for item in sublist]
    return csv_list

def csv_list_to_dfs(csv_list):
    dfs = {}
    for csv_file in csv_list:
        dfs[csv_file.stem] = pd.read_csv(csv_file)
    return dfs

def merge_dfs_by_epoch(dfs, metric, metric_rename, wide_long, smooth=True, smooth_par=0.9):
    new_df = pd.DataFrame()
    smooth_df = pd.DataFrame()
    exp_variations = []
    for name, df in dfs.items():
        if metric not in name:
            continue
        try:
            new_df['epoch']
        except KeyError:
            new_df['epoch'] = df.index + 1
            smooth_df['epoch'] = df.index + 1
        idx1 = name.find('_e') + 1
        idx2 = name.find('_2')
        exp_variation = name[idx1:idx2]
        exp_variations.append(exp_variation)
        col_name = name
        new_df[col_name] = df['Value']
        if smooth:
            smooth_df[col_name] = df['Value'].ewm(alpha=(1 - smooth_par)).mean()            
    if wide_long == 'long':
        #return new_df
        new_df = pd.melt(new_df, id_vars=['epoch'], value_name=metric_rename, var_name='exp_variation_id')
        if smooth:
            smooth_df = pd.melt(smooth_df, id_vars='epoch', value_name=metric, var_name='exp_variation_id')
            new_df[metric_rename + '-smooth'] = smooth_df[metric]
            
        new_df.loc[new_df['exp_variation_id'].str.contains('train'), 'train_val'] = 'train'
        new_df.loc[new_df['exp_variation_id'].str.contains('val'), 'train_val'] = 'val'

        new_df.loc[new_df['exp_variation_id'].str.contains('WV02'), 'sensor'] = 'WV02'
        new_df.loc[new_df['exp_variation_id'].str.contains('GE01'), 'sensor'] = 'GE01'
        new_df['sensor'].fillna('WV02', inplace=True)

        for exp_variation in exp_variations:
            label = exp_variation[exp_variation.find('-') + 1] + ' bands'
            new_df.loc[new_df['exp_variation_id'].str.contains(exp_variation), 'exp_variation'] = exp_variation
            new_df.loc[new_df['exp_variation_id'].str.contains(exp_variation), 'label'] = label
        # new_df['legend'] = new_df['exp_variation'] + '-' + new_df['sensor']
        #new_df.set_index(['exp_variation', 'train_val', 'sensor'], inplace=True)
        new_df['metric'] = metric_rename
    return new_df

## e01 Pretrain metrics plot

In [144]:
CSV_DIR = 'logs/csv'
METRICS_NAME = ['PSNR', 'SSIM', 'loss'] # The term used in tensorboard logs
METRICS = ['PSNR', 'SSIM', 'MAE'] # Change to this term instead
SMOOTH = True
SMOOTH_PAR = 0.9

In [145]:
csv_list = list_exp_csv(CSV_DIR, 'e01', 
                        pre_gan='pre',
                        metrics=METRICS_NAME)
dfs = []
for i in range(len(METRICS)):
    dfs.append(merge_dfs_by_epoch(csv_list_to_dfs(csv_list), 
                                   metric=METRICS_NAME[i], 
                                   metric_rename=METRICS[i],
                                   wide_long='long', 
                                   smooth=SMOOTH, 
                                   smooth_par=SMOOTH_PAR))
metric_df = dfs[0]
for i in range(len(METRICS) - 1):
    metric_df = metric_df.merge(dfs[i+1], how='outer')
metric_df

,epoch,exp_variation_id,PSNR,PSNR-smooth,train_val,sensor,exp_variation,label,metric,SSIM,SSIM-smooth,MAE,MAE-smooth
0,1,run-tb_e01-3-pre_20210124-153415_train-tag-epo...,16.337782,16.337782,train,WV02,e01-3-pre,3 bands,PSNR,NaN,NaN,NaN,NaN
1,2,run-tb_e01-3-pre_20210124-153415_train-tag-epo...,25.686089,21.257943,train,WV02,e01-3-pre,3 bands,PSNR,NaN,NaN,NaN,NaN
2,3,run-tb_e01-3-pre_20210124-153415_train-tag-epo...,28.110910,23.786713,train,WV02,e01-3-pre,3 bands,PSNR,NaN,NaN,NaN,NaN
3,4,run-tb_e01-3-pre_20210124-153415_train-tag-epo...,29.242495,25.373158,train,WV02,e01-3-pre,3 bands,PSNR,NaN,NaN,NaN,NaN
4,5,run-tb_e01-3-pre_20210124-153415_train-tag-epo...,30.221743,26.557154,train,WV02,e01-3-pre,3 bands,PSNR,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,396,run-tb_e01-8-pre_20210116-194500_val-WV02-tag-...,NaN,NaN,val,WV02,e01-8-pre,8 bands,MAE,NaN,NaN,0.023536,0.026698
11996,397,run-tb_e01-8-pre_20210116-194500_val-WV02-tag-...,NaN,NaN,val,WV02,e01-8-pre,8 bands,MAE,NaN,NaN,0.026950,0.026723
11997,398,run-tb_e01-8-pre_20210116-194500_val-WV02-tag-...,NaN,NaN,val,WV02,e01-8-pre,8 bands,MAE,NaN,NaN,0.028569,0.026908
11998,399,run-tb_e01-8-pre_20210116-194500_val-WV02-tag-...,NaN,NaN,val,WV02,e01-8-pre,8 bands,MAE,NaN,NaN,0.026941,0.026911


In [320]:
if SMOOTH:
    y = [metric + '-smooth' for metric in METRICS]
else:
    y = METRICS
fig = px.line(metric_df, 
              x='epoch', 
              y=y, 
              color='label', 
              #range_y=(29,44), 
              title='Experiment 01 Pretraining - Metrics plots', 
              facet_col='train_val',
              facet_row='metric',
              facet_col_spacing=0.01,
              line_dash='sensor', 
              line_dash_sequence=['solid', 'dot'], 
              render_mode='svg',
              width=1000,
              height=1000
             )
fig.update_layout(legend_title_text='')
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_yaxes(matches=None)
fig.layout.yaxis1.range = [0.005, 0.035]
fig.layout.yaxis2.range = [0.005, 0.035]
fig.layout.yaxis3.range = [0.7, 1]
fig.layout.yaxis4.range = [0.7, 1]
fig.layout.yaxis5.range = [30, 44]
fig.layout.yaxis6.range = [30, 44]
fig.layout.yaxis1.title=''
fig.layout.yaxis3.title=''
fig.layout.yaxis5.title=''
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="top",
    y=-0.05,
    xanchor="left",
    x=0
))

fig.show()

In [147]:
fig.write_image("results-public/loss-plots/loss-e01-pre.pdf")

## e01 GAN training metrics plot

In [7]:
CSV_DIR = 'logs/csv'
METRICS_NAME = ['PSNR', 'SSIM', 'D_loss', 'G_generator', 'G_perceptual', 'G_pixel', 'G_loss_total', 
                'Ma', 'NIQE', 'PI'] # The term used in tensorboard logs
METRICS = ['PSNR', 'SSIM', 'D_loss', 'G_generator', 'G_perceptual', 'G_pixel', 'G_loss_total', 
           'Ma', 'NIQE', 'PI'] # Change to this term instead
SMOOTH = True
SMOOTH_PAR = 0.9

In [8]:
csv_list = list_exp_csv(CSV_DIR, 'e01', 
                        pre_gan='gan',
                        metrics=METRICS_NAME)
dfs = []
for i in range(len(METRICS)):
    dfs.append(merge_dfs_by_epoch(csv_list_to_dfs(csv_list), 
                                   metric=METRICS_NAME[i], 
                                   metric_rename=METRICS[i],
                                   wide_long='long', 
                                   smooth=SMOOTH, 
                                   smooth_par=SMOOTH_PAR))
metric_df = dfs[0]
for i in range(len(METRICS) - 1):
    metric_df = metric_df.merge(dfs[i+1], how='outer')
metric_df

,epoch,exp_variation_id,PSNR,PSNR-smooth,train_val,sensor,exp_variation,label,metric,SSIM,...,G_pixel,G_pixel-smooth,G_loss_total,G_loss_total-smooth,Ma,Ma-smooth,NIQE,NIQE-smooth,PI,PI-smooth
0,1,run-tb_e01-8-gan_20210129-130702_train-tag-epo...,41.112080,41.112080,train,WV02,e01-8-gan,8 bands,PSNR,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,run-tb_e01-8-gan_20210129-130702_train-tag-epo...,41.088020,41.099417,train,WV02,e01-8-gan,8 bands,PSNR,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,run-tb_e01-8-gan_20210129-130702_train-tag-epo...,40.795033,40.987098,train,WV02,e01-8-gan,8 bands,PSNR,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,run-tb_e01-8-gan_20210129-130702_train-tag-epo...,40.741882,40.915794,train,WV02,e01-8-gan,8 bands,PSNR,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,run-tb_e01-8-gan_20210129-130702_train-tag-epo...,40.839371,40.897132,train,WV02,e01-8-gan,8 bands,PSNR,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3395,196,run-tb_e01-8-gan_20210129-130702_val-WV02-tag-...,NaN,NaN,val,WV02,e01-8-gan,8 bands,PI,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.098695,5.207443
3396,197,run-tb_e01-8-gan_20210129-130702_val-WV02-tag-...,NaN,NaN,val,WV02,e01-8-gan,8 bands,PI,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.181020,5.204801
3397,198,run-tb_e01-8-gan_20210129-130702_val-WV02-tag-...,NaN,NaN,val,WV02,e01-8-gan,8 bands,PI,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.347933,5.219114
3398,199,run-tb_e01-8-gan_20210129-130702_val-WV02-tag-...,NaN,NaN,val,WV02,e01-8-gan,8 bands,PI,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.606370,5.257840


In [9]:
if SMOOTH:
    y = [metric + '-smooth' for metric in METRICS]
else:
    y = METRICS

fig = px.line(metric_df, 
              x='epoch', 
              y=y, 
              color='label', 
              #range_y=(29,44), 
              title='Experiment 01 GAN training - Metrics plots', 
              facet_col='train_val',
              facet_row='metric',
              facet_col_spacing=0.01,
              line_dash='sensor', 
              line_dash_sequence=['solid', 'dot'], 
              render_mode='svg',
              width=1000,
              height=1000
             )
fig.update_layout(legend_title_text='')
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_yaxes(matches=None)
#fig.layout.yaxis1.range = [0.005, 0.035]
#fig.layout.yaxis2.range = [0.005, 0.035]
#fig.layout.yaxis3.range = [0.7, 1]
#fig.layout.yaxis4.range = [0.7, 1]
#fig.layout.yaxis5.range = [30, 44]
#fig.layout.yaxis6.range = [30, 44]
fig.layout.yaxis1.title=''
fig.layout.yaxis3.title=''
fig.layout.yaxis5.title=''
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="top",
    y=-0.05,
    xanchor="left",
    x=0
))

fig.show()

## New form of logging

In [357]:
CSV_DIRS = {'e01-8-WV02': 'logs/e01-8/csv/WV02', 
            'e01-6-WV02': 'logs/e01-6/csv/WV02', 
            'e01-4-WV02': 'logs/e01-4/csv/WV02', 
            'e01-4-GE01': 'logs/e01-4/csv/GE01', 
            'e01-3-WV02': 'logs/e01-3/csv/WV02', 
            'e01-3-GE01': 'logs/e01-3/csv/GE01', 
           }
CSV_DIRS = {'e01-8-WV02': 'logs/e01-8/csv/WV02'}
STATISTICS = ['mean', 'median', 'std', 'quantile-0.40', 'quantile-0.60']
METRICS = ['PSNR', 'SSIM']

def extract_epoch_number(pathlib_path):
    return int(pathlib_path.stem[-3:].replace('-', ''))

def list_csv(csv_dir, epoch_start=1):
    if isinstance(csv_dir, str):
        csv_dir = pathlib.Path(csv_dir)
    csv_list = list(csv_dir.glob('**/*.csv'))
    csv_dict = {}
    for csv in csv_list:
        epoch = extract_epoch_number(csv)
        if epoch_start != 1:
            epoch = epoch - epoch_start + 1
        csv_dict[epoch] = csv
    return csv_dict

def csvs_to_dfs(csv_dict):
    n = len(csv_dict)
    df_dict = {}
    for i in range(1, n + 1):
        df_dict[i] = pd.read_csv(csv_dict[i], index_col=0)
    return df_dict
def epoch_df(dfs_dict, metrics_filter, statistic='mean'):
    n = len(dfs_dict)
    epoch_dict = {}
    for i in range(1, n + 1):
        if statistic == 'mean':
            epoch_dict[i] = dfs_dict[i].loc[:,metrics_filter].mean()
        elif statistic == 'median':
            epoch_dict[i] = dfs_dict[i].loc[:,metrics_filter].median()
        elif statistic == 'std':
            epoch_dict[i] = dfs_dict[i].loc[:,metrics_filter].std()
        elif 'quantile' in statistic:
            level = float(statistic[-4:])
            epoch_dict[i] = dfs_dict[i].loc[:,metrics_filter].quantile(level)
        else:
            raise ValueError()
    epoch_df = pd.DataFrame.from_dict(epoch_dict).transpose()
    return epoch_df

def collect_all_exp_dfs(csv_dirs, statistics, metrics, epoch_start=1):
    df_dict_exp = {}
    for exp_variation in csv_dirs.keys():
        csv_dir = csv_dirs[exp_variation]
        df_dict_stat = {}
        for statistic in STATISTICS:
            df_dict_stat[statistic] = epoch_df(csvs_to_dfs(list_csv(csv_dir, epoch_start=epoch_start)), 
                                               metrics_filter=metrics, 
                                               statistic=statistic)
        df_dict_exp[exp_variation] = df_dict_stat
    return df_dict_exp
        
df_dict_exp = collect_all_exp_dfs(csv_dirs=CSV_DIRS, 
                                  statistics=STATISTICS, 
                                  metrics=METRICS, 
                                  epoch_start=1)

In [358]:
def plot_trace(fig, df_dict, metric, row, col):
    fig.add_trace(go.Scatter(
        x=df_dict[STATISTICS[-1]].index,
        y=df_dict[STATISTICS[-1]][metric],
        showlegend=False, 
        mode='lines',
        line=dict(width=0.0),
        fillcolor='blue'
    ), row=row, col=col)

    fig.add_trace(go.Scatter(
        x=df_dict[STATISTICS[-2]].index,
        y=df_dict[STATISTICS[-2]][metric],
        fill='tonexty',
        showlegend=False,
        mode='lines',
        line=dict(width=0.0),
        fillcolor='rgb(90, 180, 240)'
    ), row=row, col=col)

    fig.add_trace(go.Scatter(
        x=df_dict['median'].index,
        y=df_dict['median'][metric],
        mode='lines',
        line=dict(color='rgb(31, 119, 180)'),
        name='median'
    ), row=row, col=col)

def plot_pretrain(df_dict_exp):
    fig = make_subplots(rows=2, cols=2, 
                        subplot_titles=("Training set","Validation set"))
    for exp_variation in df_dict_exp.keys():
        df_dict_stat = df_dict_exp[exp_variation]
        plot_trace(fig, df_dict_stat, 'PSNR', row=1, col=2)
    fig.update_layout(title='Pretrain', height=800, width=800)
    fig.show()
plot_pretrain(df_dict_exp)